In [ ]:
# Hyperparameter grids
random_state_grid = [251, 582, 591, 825, 189, 888, 395, 943, 616, 896]
n_estimators_grid = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth_grid = [5, 7, 10, 12, None]
n_features = df.shape[1] - 1
max_features_grid = ['sqrt', 'log2', n_features//3, n_features//2]
reg_param_grid = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

# Store MSE and hyperparameter combinations
rmse_results_rf = []
rmse_results_depth = []
rmse_results_features = []
rmse_results_hsrf = []

mae_results_rf = []
mae_results_depth = []
mae_results_features = []
mae_results_hsrf = []

mad_results_rf = []
mad_results_depth = []
mad_results_features = []
mad_results_hsrf = []

# Run RandomForestRegressor with varying max_features
for random_state, n_estimators in itertools.product(
        random_state_grid, n_estimators_grid):

    rf_model = RandomForestRegressor(n_estimators=n_estimators,
                                         random_state=random_state, n_jobs=16, max_features=n_features//3)

    predictions_rf = []
    actual_values_rf = []

    tscv = TimeSeriesSplit(n_splits=30,  gap=0)

    for train_index, test_index in tscv.split(df):
        train, test = df.iloc[train_index], df.iloc[test_index]
        X_train, y_train = train.drop(columns=['CPIAUCSL']), train['CPIAUCSL']
        X_test, y_test = test.drop(columns=['CPIAUCSL']), test['CPIAUCSL']

        rf_model.fit(X_train, y_train)
        y_pred_rf = rf_model.predict(X_test)

        predictions_rf.extend(y_pred_rf)
        actual_values_rf.extend(y_test)

    rmse_rf = np.sqrt(mean_squared_error(actual_values_rf, predictions_rf))
    rmse_results_rf.append([random_state, n_estimators,  rmse_rf])

    mae_rf = mean_absolute_error(actual_values_rf, predictions_rf)
    mae_results_rf.append([random_state, n_estimators,  mae_rf])

    mad_rf = np.median(np.abs((np.array(actual_values_rf) - np.array(predictions_rf)) - np.median(np.array(actual_values_rf) - np.array(predictions_rf))))
    mad_results_rf.append([random_state, n_estimators,  mad_rf])

    # Print results
    print(f'RF - Random state: {random_state}, Number of estimators: {n_estimators},  RMSE: {rmse_rf}')

# Run RandomForestRegressor with varying max_depth
for random_state, n_estimators in itertools.product(
        random_state_grid, n_estimators_grid):
    for max_depth in max_depth_grid:
        rf_model_depth = RandomForestRegressor(n_estimators=n_estimators,
                                               random_state=random_state, n_jobs=16, max_depth=max_depth, max_features=n_features//3)

        predictions_depth = []
        actual_values_depth = []

        tscv = TimeSeriesSplit(n_splits=30,  gap=0)

        for train_index, test_index in tscv.split(df):
            train, test = df.iloc[train_index], df.iloc[test_index]
            X_train, y_train = train.drop(columns=['CPIAUCSL']), train['CPIAUCSL']
            X_test, y_test = test.drop(columns=['CPIAUCSL']), test['CPIAUCSL']

            rf_model_depth.fit(X_train, y_train)
            y_pred_depth = rf_model_depth.predict(X_test)

            predictions_depth.extend(y_pred_depth)
            actual_values_depth.extend(y_test)

        rmse_depth = np.sqrt(mean_squared_error(actual_values_depth, predictions_depth))
        rmse_results_depth.append([random_state, n_estimators, max_depth,  rmse_depth])

        mae_depth = mean_absolute_error(actual_values_depth, predictions_depth)
        mae_results_depth.append([random_state, n_estimators, max_depth,  mae_depth])

        mad_depth = np.median(np.abs((np.array(actual_values_depth) - np.array(predictions_depth)) - np.median(np.array(actual_values_depth) - np.array(predictions_depth))))
        mad_results_depth.append([random_state, n_estimators, max_depth,  mad_depth])

        # Print results
        print(f'RF Depth - Random state: {random_state}, Number of estimators: {n_estimators}, Max depth: {max_depth},  RMSE: {rmse_depth}')

# Run RandomForestRegressor with varying max_features
for random_state, n_estimators in itertools.product(
        random_state_grid, n_estimators_grid):
    for max_features in max_features_grid:
        rf_model_features = RandomForestRegressor(n_estimators=n_estimators,
                                                  random_state=random_state, n_jobs=16, max_features=max_features)

        predictions_features = []
        actual_values_features = []

        tscv = TimeSeriesSplit(n_splits=30,  gap=0)

        for train_index, test_index in tscv.split(df):
            train, test = df.iloc[train_index], df.iloc[test_index]
            X_train, y_train = train.drop(columns=['CPIAUCSL']), train['CPIAUCSL']
            X_test, y_test = test.drop(columns=['CPIAUCSL']), test['CPIAUCSL']

            rf_model_features.fit(X_train, y_train)
            y_pred_features = rf_model_features.predict(X_test)

            predictions_features.extend(y_pred_features)
            actual_values_features.extend(y_test)

        rmse_features = np.sqrt(mean_squared_error(actual_values_features, predictions_features))
        rmse_results_features.append([random_state, n_estimators, max_features, rmse_features])

        mae_features = mean_absolute_error(actual_values_features, predictions_features)
        mae_results_features.append([random_state, n_estimators, max_features, mae_features])

        mad_features = np.median(np.abs((np.array(actual_values_features) - np.array(predictions_features)) - np.median(np.array(actual_values_features) - np.array(predictions_features))))
        mad_results_features.append([random_state, n_estimators, max_features, mad_features])

        # Print results
        print(f'RF Features - Random state: {random_state}, Number of estimators: {n_estimators}, Max features: {max_features}, RMSE: {rmse_features}')

# Run HSTreeRegressor with varying reg_param
for random_state, n_estimators in itertools.product(
        random_state_grid, n_estimators_grid):
    for reg_param in reg_param_grid:
        rf_model_hsrf = HSTreeRegressor(estimator_=RandomForestRegressor(n_estimators=n_estimators,
                                                                        random_state=random_state, n_jobs=16, max_features=n_features//3),
                                       reg_param=reg_param, shrinkage_scheme_='node_based')

        predictions_hsrf = []
        actual_values_hsrf = []

        tscv = TimeSeriesSplit(n_splits=30,  gap=0)

        for train_index, test_index in tscv.split(df):
            train, test = df.iloc[train_index], df.iloc[test_index]
            X_train, y_train = train.drop(columns=['CPIAUCSL']), train['CPIAUCSL']
            X_test, y_test = test.drop(columns=['CPIAUCSL']), test['CPIAUCSL']

            rf_model_hsrf.fit(X_train, y_train)
            y_pred_hsrf = rf_model_hsrf.predict(X_test)

            predictions_hsrf.extend(y_pred_hsrf)
            actual_values_hsrf.extend(y_test)

        rmse_hsrf = np.sqrt(mean_squared_error(actual_values_hsrf, predictions_hsrf))
        rmse_results_hsrf.append([random_state, n_estimators,  reg_param, rmse_hsrf])

        mae_hsrf = mean_absolute_error(actual_values_hsrf, predictions_hsrf)
        mae_results_hsrf.append([random_state, n_estimators,  reg_param, mae_hsrf])

        # Print results
        print(f'HSRF - Random state: {random_state}, Number of estimators: {n_estimators},  Reg param: {reg_param}, RMSE: {rmse_hsrf}')




In [ ]:
rmse_values_rf = pd.DataFrame(rmse_results_rf, columns=['random_state', 'n_estimators', 'rmse_rf'])
rmse_values_depth = pd.DataFrame(rmse_results_depth, columns=['random_state', 'n_estimators', 'max_depth', 'rmse_depth'])
rmse_values_features = pd.DataFrame(rmse_results_features, columns=['random_state', 'n_estimators', 'max_features', 'rmse_features'])
rmse_values_hsrf = pd.DataFrame(rmse_results_hsrf, columns=['random_state', 'n_estimators', 'reg_param', 'rmse_hsrf'])

mae_values_rf = pd.DataFrame(mae_results_rf, columns=['random_state', 'n_estimators', 'mae_rf'])
mae_values_depth = pd.DataFrame(mae_results_depth, columns=['random_state', 'n_estimators', 'max_depth', 'mae_depth'])
mae_values_features = pd.DataFrame(mae_results_features, columns=['random_state', 'n_estimators', 'max_features', 'mae_features'])
mae_values_hsrf = pd.DataFrame(mae_results_hsrf, columns=['random_state', 'n_estimators', 'reg_param', 'mae_hsrf'])

mad_values_rf = pd.DataFrame(mad_results_rf, columns=['random_state', 'n_estimators', 'mad_rf'])
mad_values_depth = pd.DataFrame(mad_results_depth, columns=['random_state', 'n_estimators', 'max_depth', 'mad_depth'])
mad_values_features = pd.DataFrame(mad_results_features, columns=['random_state', 'n_estimators', 'max_features', 'mad_features'])

# store to csv
rmse_values_rf.to_csv('rmse_values_rf.csv')
rmse_values_depth.to_csv('rmse_values_depth.csv')
rmse_values_features.to_csv('rmse_values_features.csv')
rmse_values_hsrf.to_csv('rmse_values_hsrf.csv')

mae_values_rf.to_csv('mae_values_rf.csv')
mae_values_depth.to_csv('mae_values_depth.csv')
mae_values_features.to_csv('mae_values_features.csv')
mae_values_hsrf.to_csv('mae_values_hsrf.csv')

mad_values_rf.to_csv('mad_values_rf.csv')
mad_values_depth.to_csv('mad_values_depth.csv')
mad_values_features.to_csv('mad_values_features.csv')

In [ ]:
# Reading RMSE values
rmse_values_rf = pd.read_csv('rmse_values_rf.csv')
rmse_values_depth = pd.read_csv('rmse_values_depth.csv')
rmse_values_features = pd.read_csv('rmse_values_features.csv')
rmse_values_hsrf = pd.read_csv('rmse_values_hsrf.csv')

# Reading MAE values
mae_values_rf = pd.read_csv('mae_values_rf.csv')
mae_values_depth = pd.read_csv('mae_values_depth.csv')
mae_values_features = pd.read_csv('mae_values_features.csv')
mae_values_hsrf = pd.read_csv('mae_values_hsrf.csv')

# Reading MAD values
mad_values_rf = pd.read_csv('mad_values_rf.csv')
mad_values_depth = pd.read_csv('mad_values_depth.csv')
mad_values_features = pd.read_csv('mad_values_features.csv')

In [ ]:
# Fill the values of max_depth without an input with 'None'
rmse_values_depth['max_depth'].fillna('None', inplace=True)


# Compute the mean RMSE by max_depth and n_estimators
mean_rmse_depth_n_estimators = rmse_values_depth.groupby(['max_depth', 'n_estimators'])['rmse_depth'].mean()
sem_rmse_depth_n_estimators = rmse_values_depth.groupby(['max_depth', 'n_estimators'])['rmse_depth'].sem()

# for each n_estimators, give the lowest mean RMSE by max_depth
min_rmse_depth_values = mean_rmse_depth_n_estimators.groupby('n_estimators').min()

idxmin_rmse_depth_values = mean_rmse_depth_n_estimators.groupby('n_estimators').idxmin()

sem_rmse_depth_n_estimators = sem_rmse_depth_n_estimators.loc[idxmin_rmse_depth_values]



# Compute the mean RMSE by max_features
rmse_by_features = rmse_values_features.groupby('max_features')['rmse_features'].mean()
rmse_min_features = rmse_by_features.idxmin()

# Compute the mean RMSE by reg_param
rmse_by_reg_param = rmse_values_hsrf.groupby('reg_param')['rmse_hsrf'].mean()
rmse_min_reg_param = rmse_by_reg_param.idxmin()

# Compute mean RMSE by n_estimators
mean_rmse_rf = rmse_values_rf.groupby('n_estimators')['rmse_rf'].mean()

# Compute mean RMSE by n_estimators for max_depth = rmse_min_depth
mean_rmse_depth = min_rmse_depth_values

# Compute mean RMSE by n_estimators for max_features = rmse_min_features
mean_rmse_features = rmse_values_features.loc[rmse_values_features['max_features'] == rmse_min_features].groupby('n_estimators')['rmse_features'].mean()

# Compute mean RMSE by n_estimators for reg_param = rmse_min_reg_param
mean_rmse_hsrf = rmse_values_hsrf.loc[rmse_values_hsrf['reg_param'] == rmse_min_reg_param].groupby('n_estimators')['rmse_hsrf'].mean()

# Compute standard error of the mean RMSE by n_estimators
sem_rmse_rf = rmse_values_rf.groupby('n_estimators')['rmse_rf'].sem()

# Compute standard error of the mean RMSE by n_estimators for max_depth = rmse_min_depth
sem_rmse_depth = sem_rmse_depth_n_estimators

# Compute standard error of the mean RMSE by n_estimators for max_features = rmse_min_features
sem_rmse_features = rmse_values_features.loc[rmse_values_features['max_features'] == rmse_min_features].groupby('n_estimators')['rmse_features'].sem()

# Compute standard error of the mean RMSE by n_estimators for reg_param = rmse_min_reg_param
sem_rmse_hsrf = rmse_values_hsrf.loc[rmse_values_hsrf['reg_param'] == rmse_min_reg_param].groupby('n_estimators')['rmse_hsrf'].sem

In [ ]:
# Plot mean RMSE by n_estimators
# Create error bars using standard error of the mean

plt.errorbar(x=mean_rmse_rf.index, y=mean_rmse_rf, yerr=sem_rmse_rf, fmt='o', color='black', ecolor='lightgray', elinewidth=3, capsize=0, label='RF')
plt.errorbar(x=mean_rmse_hsrf.index, y=mean_rmse_hsrf, yerr=sem_rmse_hsrf, fmt='o', color='red', ecolor='lightgray', elinewidth=3, capsize=0, label='HSRF')
plt.errorbar(x=mean_rmse_depth.index, y=mean_rmse_depth, yerr=sem_rmse_depth, fmt='o', color='blue', ecolor='lightgray', elinewidth=3, capsize=0, label='RF - depth')
plt.errorbar(x=mean_rmse_features.index, y=mean_rmse_features, yerr=sem_rmse_features, fmt='o', color='green', ecolor='lightgray', elinewidth=3, capsize=0, label='RF - features')

# Adding lines between the dots
plt.plot(mean_rmse_rf.index, mean_rmse_rf, color='black', linestyle='--', linewidth=2)
plt.plot(mean_rmse_hsrf.index, mean_rmse_hsrf, color='red', linestyle='dotted', linewidth=2)
plt.plot(mean_rmse_depth.index, mean_rmse_depth, color='blue', linestyle='--', linewidth=2)
plt.plot(mean_rmse_features.index, mean_rmse_features, color='green', linestyle='-', linewidth=2)

plt.legend(loc='upper right')

plt.xlabel('Number of trees')
plt.ylabel('RMSE')

plt.show()

In [ ]:
# Fill the values of max_depth without an input with 'None'
mae_values_depth['max_depth'].fillna('None', inplace=True)

# Compute the mean MAE by max_depth and n_estimators
mean_mae_depth_n_estimators = mae_values_depth.groupby(['max_depth', 'n_estimators'])['mae_depth'].mean()
sem_mae_depth_n_estimators = mae_values_depth.groupby(['max_depth', 'n_estimators'])['mae_depth'].sem()

# for each n_estimators, give the lowest mean MAE by max_depth
min_mae_depth_values = mean_mae_depth_n_estimators.groupby('n_estimators').min()

idxmin_mae_depth_values = mean_mae_depth_n_estimators.groupby('n_estimators').idxmin()

sem_mae_depth_n_estimators = sem_mae_depth_n_estimators.loc[idxmin_mae_depth_values]

# Compute the mean MAE by max_features
mae_by_features = mae_values_features.groupby('max_features')['mae_features'].mean()
mae_min_features = mae_by_features.idxmin()

# Compute the mean MAE by reg_param
mae_by_reg_param = mae_values_hsrf.groupby('reg_param')['mae_hsrf'].mean()
mae_min_reg_param = mae_by_reg_param.idxmin()

# Compute mean MAE by n_estimators
mean_mae_rf = mae_values_rf.groupby('n_estimators')['mae_rf'].mean()

# Compute mean MAE by n_estimators for max_depth = mae_min_depth
mean_mae_depth = min_mae_depth_values

# Compute mean MAE by n_estimators for max_features = mae_min_features
mean_mae_features = mae_values_features.loc[mae_values_features['max_features'] == mae_min_features].groupby('n_estimators')['mae_features'].mean()

# Compute mean MAE by n_estimators for reg_param = mae_min_reg_param
mean_mae_hsrf = mae_values_hsrf.loc[mae_values_hsrf['reg_param'] == mae_min_reg_param].groupby('n_estimators')['mae_hsrf'].mean()

# Compute standard error of the mean MAE by n_estimators
sem_mae_rf = mae_values_rf.groupby('n_estimators')['mae_rf'].sem()

# Compute standard error of the mean MAE by n_estimators for max_depth = mae_min_depth
sem_mae_depth = sem_mae_depth_n_estimators

# Compute standard error of the mean MAE by n_estimators for max_features = mae_min_features
sem_mae_features = mae_values_features.loc[mae_values_features['max_features'] == mae_min_features].groupby('n_estimators')['mae_features'].sem()

# Compute standard error of the mean MAE by n_estimators for reg_param = mae_min_reg_param
sem_mae_hsrf = mae_values_hsrf.loc[mae_values_hsrf['reg_param'] == mae_min_reg_param].groupby('n_estimators')['mae_hsrf'].sem()


In [ ]:
# Plot mean MAE by n_estimators
# Create error bars using standard error of the mean

plt.errorbar(x=mean_mae_rf.index, y=mean_mae_rf, yerr=sem_mae_rf, fmt='o', color='black', ecolor='lightgray', elinewidth=3, capsize=0, label='RF')
plt.errorbar(x=mean_mae_hsrf.index, y=mean_mae_hsrf, yerr=sem_mae_hsrf, fmt='o', color='red', ecolor='lightgray', elinewidth=3, capsize=0, label='HSRF')
plt.errorbar(x=mean_mae_depth.index, y=mean_mae_depth, yerr=sem_mae_depth, fmt='o', color='blue', ecolor='lightgray', elinewidth=3, capsize=0, label='RF - depth')
plt.errorbar(x=mean_mae_features.index, y=mean_mae_features, yerr=sem_mae_features, fmt='o', color='green', ecolor='lightgray', elinewidth=3, capsize=0, label='RF - features')

# Adding lines between the dots
plt.plot(mean_mae_rf.index, mean_mae_rf, color='black', linestyle='--', linewidth=2)
plt.plot(mean_mae_hsrf.index, mean_mae_hsrf, color='red', linestyle='dotted', linewidth=2)
plt.plot(mean_mae_depth.index, mean_mae_depth, color='blue', linestyle='--', linewidth=2)
plt.plot(mean_mae_features.index, mean_mae_features, color='green', linestyle='-', linewidth=2)

plt.legend(loc='upper right')

plt.xlabel('Number of trees')
plt.ylabel('MAE')

plt.show()


In [ ]:
mad_values_depth['max_depth'].fillna('None', inplace=True)

# Compute the mean MAD by max_depth and n_estimators
mean_mad_depth_n_estimators = mad_values_depth.groupby(['max_depth', 'n_estimators'])['mad_depth'].mean()
sem_mad_depth_n_estimators = mad_values_depth.groupby(['max_depth', 'n_estimators'])['mad_depth'].sem()

# for each n_estimators, give the lowest mean MAD by max_depth
min_mad_depth_values = mean_mad_depth_n_estimators.groupby('n_estimators').min()

idxmin_mad_depth_values = mean_mad_depth_n_estimators.groupby('n_estimators').idxmin()

sem_mad_depth_n_estimators = sem_mad_depth_n_estimators.loc[idxmin_mad_depth_values]


# Compute the mean MAD by max_features
mad_by_features = mad_values_features.groupby('max_features')['mad_features'].mean()
mad_min_features = mad_by_features.idxmin()



# Compute mean MAD by n_estimators
mean_mad_rf = mad_values_rf.groupby('n_estimators')['mad_rf'].mean()

# Compute mean MAD by n_estimators for max_depth = mad_min_depth
mean_mad_depth = min_mad_depth_values

# Compute mean MAD by n_estimators for max_features = mad_min_features
mean_mad_features = mad_values_features.loc[mad_values_features['max_features'] == mad_min_features].groupby('n_estimators')['mad_features'].mean()


# Compute standard error of the mean MAD by n_estimators
sem_mad_rf = mad_values_rf.groupby('n_estimators')['mad_rf'].sem()

# Compute standard error of the mean MAD by n_estimators for max_depth = mad_min_depth
sem_mad_depth = sem_mad_depth_n_estimators

# Compute standard error of the mean MAD by n_estimators for max_features = mad_min_features
sem_mad_features = mad_values_features.loc[mad_values_features['max_features'] == mad_min_features].groupby('n_estimators')['mad_features'].sem()



In [ ]:
plt.errorbar(x=mean_mad_rf.index, y=mean_mad_rf, yerr=sem_mad_rf, fmt='o', color='black', ecolor='lightgray', elinewidth=3, capsize=0, label='RF')
plt.errorbar(x=mean_mad_rf.index, y=mean_mad_rf, yerr=sem_mad_rf, fmt='o', color='red', ecolor='lightgray', elinewidth=3, capsize=0, label='HSRF')
plt.errorbar(x=mean_mad_depth.index, y=mean_mad_depth, yerr=sem_mad_depth, fmt='o', color='blue', ecolor='lightgray', elinewidth=3, capsize=0, label='RF - depth')
plt.errorbar(x=mean_mad_features.index, y=mean_mad_features, yerr=sem_mad_features, fmt='o', color='green', ecolor='lightgray', elinewidth=3, capsize=0, label='RF - features')

# Adding lines between the dots
plt.plot(mean_mad_rf.index, mean_mad_rf, color='black', linestyle='--', linewidth=2)
plt.plot(mean_mad_rf.index, mean_mad_rf, color='red', linestyle='dotted', linewidth=2)
plt.plot(mean_mad_depth.index, mean_mad_depth, color='blue', linestyle='--', linewidth=2)
plt.plot(mean_mad_features.index, mean_mad_features, color='green', linestyle='-', linewidth=2)

plt.legend(loc='upper right')

plt.xlabel('Number of trees')
plt.ylabel('MAD')

plt.show()